# 작업형1

### 문제1

In [15]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-1.csv")

In [16]:
df.head()

,소방서,출동시간,도착시간
0,소방서1,2023-09-16 06:12:29,2023-09-16 07:51:55
1,소방서5,2023-09-19 06:11:41,2023-09-19 08:08:36
2,소방서1,2023-09-14 06:12:00,2023-09-14 07:24:21
3,소방서6,2023-09-20 06:12:24,2023-09-20 06:30:45
4,소방서9,2023-09-15 06:11:50,2023-09-15 06:44:00


In [17]:
df['출동시간'] = pd.to_datetime(df['출동시간'])
df['도착시간'] = pd.to_datetime(df['도착시간'])

df['시간차'] = df['도착시간'] - df['출동시간']
df['시간차'] = df['시간차'].dt.total_seconds() / 60
df = df.groupby('소방서')['시간차'].mean().max()
round(df)

81

### 문제2

In [24]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-2.csv")

In [25]:
df.head()

,학교명,교사수,1학년,2학년,3학년,4학년,5학년,6학년
0,학교1,20,118,72,154,118,77,59
1,학교2,31,187,193,135,137,63,90
2,학교3,23,136,193,135,112,91,50
3,학교4,32,72,146,67,51,130,164
4,학교5,22,61,73,86,82,55,124


In [26]:
df['전체학생'] = df['1학년'] + df['2학년'] + df['3학년'] + df['4학년'] + df['5학년'] + df['6학년']
df['비율'] = df['전체학생'] / df['교사수']
x = df['비율'].max()
df = df[df['비율'] == x]
print(df)
print(df['교사수'][7])

   학교명  교사수  1학년  2학년  3학년  4학년  5학년  6학년  전체학생   총학생         비율
7  학교8   19  151  181  161  187  174  183  1037  2074  54.578947
19


### 문제3

In [27]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-1-3.csv")

In [28]:
df.head()

,날짜,강력범죄,절도범죄,폭력범죄,지능범죄,풍속범죄,교통범죄,경찰서명
0,2020년 01월,22,102,86,62,28,212,B경찰서
1,2020년 02월,26,138,80,61,31,183,E경찰서
2,2020년 03월,14,129,76,60,29,202,C경찰서
3,2020년 04월,26,142,83,71,33,182,B경찰서
4,2020년 05월,28,131,80,72,28,212,B경찰서


In [42]:
df['총범죄'] = df.iloc[:, 1:7].sum(axis=1)
df['연도'] = df['날짜'].str[:4]
result = df.groupby('연도')['총범죄'].sum().max()
print(round(result / 12))

533


# 작업형2

In [83]:
# 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/energy_test.csv")

In [84]:
print(train.shape, test.shape)
print(train.describe(), test.describe())
print(train.info(), test.info())
print(train.isnull().sum().sum(), test.isnull().sum().sum())

(537, 10) (231, 9)
          Compac   Surf_Area   Wall_Area  Glaze_Area  Glaze_Distr   Cool_Load
count  537.00000  537.000000  537.000000  537.000000   537.000000  537.000000
mean     0.76879  667.796089  318.271881    0.237244     2.808194   24.939628
std      0.10636   88.123635   42.447824    0.132183     1.541251    9.600170
min      0.62000  514.500000  245.000000    0.000000     0.000000   11.170000
25%      0.69000  588.000000  294.000000    0.100000     2.000000   15.770000
50%      0.76000  661.500000  318.500000    0.250000     3.000000   25.160000
75%      0.86000  735.000000  343.000000    0.400000     4.000000   33.540000
max      0.98000  808.500000  416.500000    0.400000     5.000000   48.030000            Compac   Surf_Area   Wall_Area  Glaze_Area  Glaze_Distr   Cool_Load
count  231.000000  231.000000  231.000000  231.000000   231.000000  231.000000
mean     0.753420  680.803030  319.030303    0.227706     2.822511   23.769784
std      0.103843   87.513907   46.341867 

In [85]:
from sklearn.preprocessing import LabelEncoder

target = train.pop('Heat_Load')
cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, target, random_state=0, test_size=0.2)

from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print(f1_score(y_pred, y_test, average='macro'))
      
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=0)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(f1_score(y_pred, y_test, average='macro'))

import xgboost as xgb
xg = xgb.XGBClassifier(random_state=0)
xgb_data = le.fit_transform(y_train)
xg.fit(x_train, xgb_data)
y_pred = xg.predict(x_test)
y_pred = le.inverse_transform(y_pred)
print(f1_score(y_pred, y_test, average='macro'))

import lightgbm as lgbm
lg = lgbm.LGBMClassifier(random_state=0, verbose=-1)
lg.fit(x_train, y_train)
y_pred = lg.predict(x_test)
print(f1_score(y_pred, y_test, average='macro'))

0.9489195376832467
0.8938677965604732
0.9374839068652628
0.9319703995747778


In [86]:
y_pred = lg.predict(test)
result = pd.DataFrame({
    'pred':y_pred
})
result.to_csv('result.csv', index=False)

In [88]:
result.head(), result.shape

(   pred
 0   Low
 1  High
 2  High
 3   Low
 4   Low,
 (231, 1))

In [91]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lg, train, target, cv=5, scoring='f1_macro')
print(scores)

print(scores.mean())

[0.89416785 0.9423412  0.92497696 0.94250337 0.95731908]
0.9322616906462656


# 작업형3

### 문제1

In [92]:
import pandas as pd
df = pd.DataFrame({
    "항암약":[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4]
    })

In [93]:
df.head()

,항암약
0,4
1,4
2,3
3,4
4,1


In [94]:
df_x = df[df['항암약'] == 4]
print(len(df_x) / len(df))

0.55


In [97]:
df['항암약'].value_counts(normalize=True)

항암약
4    0.55
1    0.20
2    0.15
3    0.10
Name: proportion, dtype: float64

In [99]:
from scipy.stats import chisquare
prob = [0.1, 0.05, 0.15, 0.7]
print(len(df))

expected_counts = [0.1 * 20, 0.05 * 20, 0.15 * 20, 0.7 * 20]

observed_counts = df['항암약'].value_counts().sort_index().to_list()
print(observed_counts)

20
[4, 3, 2, 11]


In [101]:
print(chisquare(f_obs=observed_counts, f_exp=expected_counts))

Power_divergenceResult(statistic=6.976190476190476, pvalue=0.07266054733847573)


### 문제2

In [102]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch6/data6-3-2.csv")

In [103]:
df.head()

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7
2,102.83,6.42,41.00,20.5
3,84.94,10.20,33.44,22.2
4,94.21,4.95,29.97,21.4


In [104]:
from statsmodels.formula.api import ols

formula = 'temperature ~ solar + wind + o3'

model = ols(formula, data=df).fit()

print(model.params['o3'])

0.07493854378136539


In [106]:
print(model.pvalues['wind'])

0.7797177202071623


In [108]:
data = pd.DataFrame({
    'solar':[100],
    'wind':[5],
    'o3':[30]
})
print(model.predict(data)[0])

21.561630469479688
